# Aggregating DataFrames in PySpark

In [2]:
%config Completer.use_jedi = False

import os
import sys
import shutil
import pandas as pd

pd.set_option('display.max_colwidth', None)

BASE_DIR = os.path.realpath(os.path.join(os.getcwd(), "..", ".."))

if not BASE_DIR in sys.path:
    sys.path.append(BASE_DIR)
    
from utils import extract_zip

DATASETS_PATH = "datasets/"

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

spark = SparkSession.builder.appName("Aggregation").getOrCreate()

# Allow use single slash when using raw string (r"...")
# spark.sql("SET spark.sql.parser.escapedStringLiterals=true")

# Fix the Date conversion problem - REVIEW THE DEFINITIVE SOLUTION LATER!!!
spark.sql("SET spark.sql.legacy.timeParserPolicy=LEGACY")

spark

In [5]:
data_file = extract_zip(
    zip_file=os.path.join(DATASETS_PATH, "nyc_air_bnb.csv.zip"),
    member="nyc_air_bnb.csv"
)

data_file

'/tmp/aaaaa-7zdpew8n/nyc_air_bnb.csv'

In [6]:
airbnb = spark.read.csv(data_file, inferSchema=True, header=True)

In [9]:
airbnb.limit(5).toPandas()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.9419,Private room,150,3,0,None,None,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


In [8]:
airbnb.printSchema()

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- host_id: string (nullable = true)
 |-- host_name: string (nullable = true)
 |-- neighbourhood_group: string (nullable = true)
 |-- neighbourhood: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- room_type: string (nullable = true)
 |-- price: string (nullable = true)
 |-- minimum_nights: string (nullable = true)
 |-- number_of_reviews: string (nullable = true)
 |-- last_review: string (nullable = true)
 |-- reviews_per_month: string (nullable = true)
 |-- calculated_host_listings_count: string (nullable = true)
 |-- availability_365: integer (nullable = true)



In [18]:
airbnb = (
    airbnb
    .withColumn("id", airbnb["id"].cast(IntegerType()))
    .withColumn("host_id", airbnb["host_id"].cast(IntegerType()))
    .withColumn("latitude", airbnb["latitude"].cast(FloatType()))
    .withColumn("longitude", airbnb["longitude"].cast(FloatType()))
    .withColumn("price", airbnb["price"].cast(FloatType()))
    .withColumn("minimum_nights", airbnb["minimum_nights"].cast(IntegerType()))
    .withColumn(
        "number_of_reviews", airbnb["number_of_reviews"].cast(IntegerType())
    )
    .withColumn(
        "last_review", to_date(col("last_review"), "yyyy-MM-dd")
    )
    .withColumn(
        "reviews_per_month", airbnb["reviews_per_month"].cast(FloatType())
    )
    .withColumn(
        "calculated_host_listings_count", 
        airbnb["calculated_host_listings_count"].cast(IntegerType())
    )
    .withColumn(
        "availability_365", airbnb["availability_365"].cast(IntegerType())
    )
)

In [19]:
airbnb.printSchema()

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- host_id: integer (nullable = true)
 |-- host_name: string (nullable = true)
 |-- neighbourhood_group: string (nullable = true)
 |-- neighbourhood: string (nullable = true)
 |-- latitude: float (nullable = true)
 |-- longitude: float (nullable = true)
 |-- room_type: string (nullable = true)
 |-- price: float (nullable = true)
 |-- minimum_nights: integer (nullable = true)
 |-- number_of_reviews: integer (nullable = true)
 |-- last_review: date (nullable = true)
 |-- reviews_per_month: float (nullable = true)
 |-- calculated_host_listings_count: integer (nullable = true)
 |-- availability_365: integer (nullable = true)



In [22]:
airbnb.groupBy("neighbourhood").agg({"price": "mean"}).show(5)

+-------------+----------+
|neighbourhood|avg(price)|
+-------------+----------+
|       Corona| 59.171875|
| Richmondtown|      78.0|
| Prince's Bay|     409.5|
|  Westerleigh|      71.5|
|   Mill Basin|    179.75|
+-------------+----------+
only showing top 5 rows



In [25]:
airbnb.groupBy(
    "neighbourhood"
).agg(
    min(airbnb.price).alias("Min"),
    max(airbnb.price).alias("Max")
).show()

+-----------------+----+------+
|    neighbourhood| Min|   Max|
+-----------------+----+------+
|           Corona|23.0| 359.0|
|     Richmondtown|78.0|  78.0|
|     Prince's Bay|85.0|1250.0|
|      Westerleigh|40.0| 103.0|
|       Mill Basin|85.0| 299.0|
|         40.76199| 1.0|   1.0|
|     Civic Center|50.0| 950.0|
|         40.83166| 1.0|   1.0|
|       Douglaston|40.0| 178.0|
|       Mount Hope|24.0| 250.0|
|          40.7578| 1.0|   1.0|
|         40.80958| 1.0|   1.0|
|      Marble Hill|40.0| 274.0|
|        Rego Park|21.0| 300.0|
|         40.81225| 2.0|   2.0|
|         40.76805| 1.0|   1.0|
|         40.64936| 1.0|   1.0|
|    Dyker Heights|30.0| 170.0|
|         40.76364| 2.0|   2.0|
|Kew Gardens Hills|40.0| 399.0|
+-----------------+----+------+
only showing top 20 rows



In [31]:
airbnb.summary(
    "count", "min", "25%", "75%", "mean", "max"
).toPandas()

,summary,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
0,count,48895,49047,48729,48873,48894,48894,48885,48736,48894,48887,48891,48738,38858,48891,48737
1,min,2539,1 Bed Apt in Utopic Williamsburg,2438,"very clean studio app""",194716858,2,-74.16254,-74.24442,-73.90783,-74.00828,0,0,0.0,0,0
2,25%,9471893,2.4544724E7,7797690,475.0,1.94716858E8,40.68771,40.68981,-73.98309,56.0,69.0,1,1,0.19,1,0
3,75%,29152899,1.74786681E8,107434423,NaN,1.97400421E8,40.78304,40.76299,-73.93638,145.0,175.0,5,23,2.01,2,226
4,mean,1.9017143236179568E7,1.02037532075E8,6.749591589946438E7,NaN,1.960586395E8,40.49195828025477,40.36302551530165,437.1107574129136,148.10106579268293,152.2229629934223,7.1286126280910596,23.25827896097501,1.3743823663657475,7.65500807919658,112.59808769518025
5,max,36487245,"ﾏﾝﾊｯﾀﾝ､駅から徒歩4分でどこに行くのにも便利な場所!女性の方希望,ｷﾚｲなお部屋｡",274321313,현선,Woodside,Woodside,40.91306,2.4906404E7,Shared room,10000.0,1250,629,58.5,365,365


In [27]:
airbnb.printSchema()

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- host_id: integer (nullable = true)
 |-- host_name: string (nullable = true)
 |-- neighbourhood_group: string (nullable = true)
 |-- neighbourhood: string (nullable = true)
 |-- latitude: float (nullable = true)
 |-- longitude: float (nullable = true)
 |-- room_type: string (nullable = true)
 |-- price: float (nullable = true)
 |-- minimum_nights: integer (nullable = true)
 |-- number_of_reviews: integer (nullable = true)
 |-- last_review: date (nullable = true)
 |-- reviews_per_month: float (nullable = true)
 |-- calculated_host_listings_count: integer (nullable = true)
 |-- availability_365: integer (nullable = true)



In [30]:
airbnb.select(
    "price", "minimum_nights", "number_of_reviews"
).summary(
    "count", "min", "25%", "75%", "mean", "max"
).show()

+-------+-----------------+------------------+-----------------+
|summary|            price|    minimum_nights|number_of_reviews|
+-------+-----------------+------------------+-----------------+
|  count|            48887|             48891|            48738|
|    min|        -74.00828|                 0|                0|
|    25%|             69.0|                 1|                1|
|    75%|            175.0|                 5|               23|
|   mean|152.2229629934223|7.1286126280910596|23.25827896097501|
|    max|          10000.0|              1250|              629|
+-------+-----------------+------------------+-----------------+



In [32]:
airbnb.agg(
    min(airbnb.price).alias("Min"),
    max(airbnb.price).alias("Max")
).show()

+---------+-------+
|      Min|    Max|
+---------+-------+
|-74.00828|10000.0|
+---------+-------+



In [35]:
airbnb.select(
    countDistinct("neighbourhood_group"), 
    avg("price"),
    stddev("price")
).show()

+-----------------------------------+-----------------+------------------+
|count(DISTINCT neighbourhood_group)|       avg(price)|stddev_samp(price)|
+-----------------------------------+-----------------+------------------+
|                                 77|152.2229629934223|238.54148640284316|
+-----------------------------------+-----------------+------------------+



In [38]:
airbnb.groupBy(
    "room_type"
).pivot(
    "neighbourhood_group", ["Queens", "Brooklyn"]
).count().show()

+-----------+------+--------+
|  room_type|Queens|Brooklyn|
+-----------+------+--------+
|         51|  null|    null|
|        205|  null|    null|
|         54|  null|    null|
|        200|  null|    null|
|        279|  null|    null|
|        138|  null|    null|
|         69|  null|    null|
|         42|  null|    null|
|Shared room|   198|     413|
|  -73.95777|  null|    null|
|        425|  null|    null|
|         59|  null|    null|
|        250|  null|    null|
|        160|  null|    null|
|   40.57453|  null|    null|
|         85|  null|    null|
|         35|  null|    null|
|  -73.95179|  null|    null|
|        410|  null|    null|
|        298|  null|    null|
+-----------+------+--------+
only showing top 20 rows



In [40]:
airbnb.filter(
    "room_type='Shared room'"
).groupBy(
    "room_type"
).pivot(
    "neighbourhood_group", ["Queens", "Brooklyn"]
).count().show()

+-----------+------+--------+
|  room_type|Queens|Brooklyn|
+-----------+------+--------+
|Shared room|   198|     413|
+-----------+------+--------+



In [42]:
airbnb.groupBy(
    "room_type"
).pivot(
    "neighbourhood_group", ["Queens", "Brooklyn"]
).agg(
    min(airbnb.price).alias("Min"),
    max(airbnb.price).alias("Max")
).show()

+-----------+----------+----------+------------+------------+
|  room_type|Queens_Min|Queens_Max|Brooklyn_Min|Brooklyn_Max|
+-----------+----------+----------+------------+------------+
|         51|      null|      null|        null|        null|
|        205|      null|      null|        null|        null|
|         54|      null|      null|        null|        null|
|        200|      null|      null|        null|        null|
|        279|      null|      null|        null|        null|
|        138|      null|      null|        null|        null|
|         69|      null|      null|        null|        null|
|         42|      null|      null|        null|        null|
|Shared room|      11.0|    1800.0|         0.0|       725.0|
|  -73.95777|      null|      null|        null|        null|
|        425|      null|      null|        null|        null|
|         59|      null|      null|        null|        null|
|        250|      null|      null|        null|        null|
|       